In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage

In [ ]:
#    latest_close_price = USEquityPricing.close.latest
#    close_price_filter = latest_close_price > 20
#    top_close_price = close_price_filter.top(5)
    
#    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
#   mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
#    mean_crossover_filter = mean_close_10 < mean_close_30

In [ ]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = (dollar_volume > 10000000)
    
    top_high_dollar_volume = dollar_volume.percentile_between(90, 100)
    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20
    is_tradeable = top_high_dollar_volume & above_20 # & (AND) | (OR) opporators.
    
    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'dollar_volume': dollar_volume,
            'high_dollar_volume': high_dollar_volume,
        },
        #screen=~high_dollar_volume, # Toggeling between True and False (~) will return > and < doll_vol securities.
        screen=is_tradeable,
    )

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)

In [ ]:
result.head()